In [3]:
from PIL import Image
import os

In [9]:
from keras.optimizers import Adam


In [ ]:
class GAN:
    def __init__(self):
        self.img_rows = 190
        self.img_cols = 135
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        optimizer = Adam(0.0002, 0.5)
        
        # Discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy',
                                  optimizer = optimizer,
                                  metrics = ['accuracy'])
        
        # Generator
        self.generator = self.build_generator()
        self.generator.compile(loss = 'binary_crossentropy', 
                               optimizer = optimizer)
        
        inp = Input(shape = (100,))
        img = self.generator(inp)
        
        self.discriminator.trainable = False
        
        